In [1]:
import random
import json
import pickle
import numpy as np 
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD


In [2]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ["?","!",".",","]

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)

[(['Hello'], 'greetings'), (['Hi'], 'greetings'), (['Hey'], 'greetings'), (['Howdy'], 'greetings'), (['How', 'are', 'you', '?'], 'greetings'), (['What', "'s", 'up', '?'], 'greetings'), (['Good', 'morning'], 'greetings'), (['Wake', 'up', '!'], 'greetings'), (['How', 'is', 'it', 'going', '?'], 'greetings'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Shut', 'up'], 'goodbye'), (['Do', "n't", 'talk'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Have', 'a', 'nice', 'day'], 'goodbye'), (['I', 'quit'], 'goodbye'), (['What', 'is', 'your', 'name', '?'], 'name'), (['What', 'can', 'I', 'call', 'you', '?'], 'name'), (['Can', 'you', 'tell', 'me', 'your', 'name', '?'], 'name'), (['Who', 'are', 'you', '?'], 'name'), (['Please', 'tell', 'me', 'your', 'name'], 'name'), (['Can', 'you', 'help', 'me', 'with', 'the', 'transaction', '?'], 'transact'), (['How', 'to', 'transact', 'money', '?'], 'transact'), (['How', 'to', 'transfer', 'money', 'to', 'someone', "'s", 'account', '?'], 'transact')

In [4]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [5]:
training = []
output_empty = [0] * len(classes)
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag,output_row])

In [6]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\sambl\AppData\Local\Temp\ipykernel_58024\4024136173.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [7]:
model = Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))

In [8]:
sgd = SGD(learning_rate=0.01,weight_decay=1e-6, momentum=0.9,nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5,verbose=None)
model.save('chatbotmodel.model')
print("Done")

INFO:tensorflow:Assets written to: chatbotmodel.model\assets


INFO:tensorflow:Assets written to: chatbotmodel.model\assets


Done


In [9]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=None)
model.save('chatbotmodel.h5',hist)
print("Done")

Done
